In [1]:
import cv2
import numpy as np
from pathlib import Path
from scipy import stats
import pandas as pd
from webcolors import rgb_to_name, hex_to_rgb

def read_image(input_path: Path) -> np.ndarray:
    """
    Read the image from the given path using OpenCV.
    """
    img = cv2.imread(str(input_path), cv2.IMREAD_UNCHANGED)
    return img

def save_image(output_path: Path, img: np.ndarray) -> None:
    """
    Save the processed image to the given path using OpenCV.
    """
    cv2.imwrite(str(output_path), img)

def calculate_most_common_color(square: np.ndarray) -> np.ndarray:
    """
    Calculate the most common color of the given square.
    """
    reshaped_square = square.reshape(-1, square.shape[-1])
    mode_color = stats.mode(reshaped_square[:, :3], axis=0)[0]
    return mode_color

def find_closest_color(color: np.ndarray, common_primary_colors: list) -> np.ndarray:
    """
    Find the closest primary color in common_primary_colors to the given color.
    """
    closest_color = common_primary_colors[np.argmin([np.linalg.norm(color - primary_color) 
                                                        for primary_color in common_primary_colors])]
    return closest_color

def draw_square(image: np.ndarray, 
                x: int, 
                y: int, 
                square_size: int, 
                border_color: tuple, 
                fill_color: tuple) -> None:
    """
    Draw a square with a specified border and 
    fill color on the image at the specified location.
    """
    image[y:y+square_size, x:x+square_size] = fill_color
    image[y:y+1, x:x+square_size] = border_color
    image[y+square_size-1:y+square_size, x:x+square_size] = border_color
    image[y:y+square_size, x:x+1] = border_color
    image[y:y+square_size, x+square_size-1:x+square_size] = border_color

def get_html_color_name(color: tuple) -> str:
    """
    Get the HTML color name for a given RGB color.
    """
    hex_color = '#{:02x}{:02x}{:02x}'.format(color[0], color[1], color[2])
    try:
        color_name = rgb_to_name(hex_to_rgb(hex_color))
    except ValueError:
        color_name = hex_color
    return color_name

def process_image(image: np.ndarray, square_size: int, common_primary_colors: list) -> np.ndarray:
    """
    Process the image by replacing each square with 
    the closest primary color and creating 
    a new image with white or light gray squares.
    """
    output_image = np.zeros_like(image)
    height, width, _ = image.shape
    color_mapping = []

    for i in range(0, height, square_size):
        for j in range(0, width, square_size):
            square = image[i:i+square_size, j:j+square_size]
            most_common_color = calculate_most_common_color(square)
            closest_primary_color = find_closest_color(most_common_color, 
                                                       common_primary_colors)
            html_color_name = get_html_color_name(tuple(closest_primary_color[:3]))
            color_mapping.append({"coordinates": (j, i), 
                                  "color": closest_primary_color.tolist(), 
                                  "color_name": html_color_name})

            # if html_color_name == "white":
            #     fill_color = (192, 192, 192, 255)  # Light gray when the color is white
            # else:
            #     fill_color = tuple(closest_primary_color[:3]) + (255,)  # Found color when it's not white

            # draw_square(output_image, j, i, square_size, (192, 192, 192, 255), fill_color)

            if html_color_name == "white":
                fill_color = (192, 192, 192, 255)
            else:
                fill_color = (255, 255, 255, 255)

            draw_square(output_image, j, i, square_size, (192, 192, 192, 255), fill_color)

    return output_image, color_mapping

def write_csv(csv_path: Path, color_mapping: list) -> None:
    """
    Write the color mapping to a CSV file.
    """
    df = pd.DataFrame(color_mapping)
    df.to_csv(csv_path, index=False)

def main():
    base_folder = Path("/Users/senthilgandhi/Dropbox/parent/workspace/pixel_puzzle_math")
    input_path = base_folder / "input_image.png"
    output_path = base_folder / "output_image.png"
    csv_path = base_folder / "color_mapping.csv"
    square_size = 16

    common_primary_colors = [
        np.array([255, 0, 0]),    # Red
        np.array([0, 255, 0]),    # Green
        np.array([0, 0, 255]),    # Blue
        np.array([255, 255, 0]),  # Yellow
        np.array([0, 255, 255]),  # Cyan
        np.array([255, 0, 255]),  # Magenta
        np.array([128, 0, 0]),    # Maroon
        np.array([0, 128, 0]),    # Dark green
        np.array([0, 0, 128]),    # Navy
        np.array([128, 128, 0])   # Olive
    ]

    input_image = read_image(input_path)
    processed_image, color_mapping = process_image(input_image, square_size, common_primary_colors)
    save_image(output_path, processed_image)
    write_csv(csv_path, color_mapping)

    df = pd.DataFrame(color_mapping)
    unique_colors = df['color_name'].nunique()
    non_white_squares = df[df['color_name'] != 'white'].shape[0]

    print(f"Unique colors encountered: {unique_colors}")
    print(f"Total number of non-white squares: {non_white_squares}")

if __name__ == "__main__":
    main()



Unique colors encountered: 7
Total number of non-white squares: 7668
